<img src="./img/logos_combined.png" align="right" width="50%"></img>
<br><br><br>

# Web Coverage Processing Service (WCPS) - Introduction and example

<hr>
<a href="./01_owslib_example.ipynb"><< OWSlib example</a>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space> <a href="./03_es_data_lib.ipynb">es-data-lib example >></a>

## WCPS request

A **Web Coverage Processing Service** is an extension supported by the WCS 2.0 core specification and can optionally be implemented by a WCS server. WCPS allows for crafting queries to be run on the data using a text based query language, similar to SQL.  
The extension specifies an additional WCS request:  
> the ***ProcessCoverages*** request returns a coverage encoded in a specified format (e.g. netCDF or csv) and allows for mathematical calculations on the requested coverages. 
>> Example: Convert a 2D global field of 2m air temperature from 31 December 2015 from K to degC  
<a href='http://earthserver.ecmwf.int/rasdaman/ows?service=WCS&version=2.0.1&request=ProcessCoverages&query=for c in (temp2m) return encode (c[ansi("2015-12-31T00:00")] - 273.15,"csv")' target="_blank">http://earthserver.ecmwf.int/rasdaman/ows?service=WCS&version=2.0.1&request=ProcessCoverages&query=for c in (temp2m) return encode (c[ansi("2015-12-31T00:00")] - 273.15,"csv")</a>

<hr>

## <a id="requests_lib"></a> requests library - Using WCS from Python

#### Retrieve a time-series of global monthly averages of 2m air temperature and store it as python list


WCS request of retrieving a time series of global monthly means. In the placeholders the parameter and the temporal limits of the time-series (in ISO-format, e.g. "2010-01-01T00:00") have to be specified. The query contains following processing steps: (1) Surface temperature values in Kelvin are converted to degree Celsius and (2) 6-hourly global time-series is averaged based on specified time-series limits.
* Access point: http://earthserver.ecmwf.int/rasdaman/ows?service=WCS&version=2.0.1
* Request type: &ProcessCoverages
* WCPS query (example): &query=for c in (temp2m) return encode(avg(c[ansi("2001-01-01T00:00":"2001-01-31T18:00")] - 273.15), "csv")

In [ ]:
url_fmt = 'http://earthserver.ecmwf.int/rasdaman/ows?service=WCS&version=2.0.1' \
          '&request=ProcessCoverages' \
          '&query=for c in (%s) return encode(avg(c[ansi("%s":"%s")] - 273.15), "csv")'

In [ ]:
# Helper lists for handling datetime objects

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# A list for every first day in the month from Jan 1979 to Dec 2014
startDateList = list((range(12)))
x =datetime(1979,1,1,0)
for i in range(len(startDateList)):
    startDateList[i]=x
    x=(x.replace(day=1)+timedelta(days=31)).replace(day=1)

# A list with every last day of the month from Jan 1979 to Dec 2014
endDateList = list((range(12)))
for i in range(0, len(endDateList)):
    y = startDateList[i].replace(hour=18) + relativedelta(day=31)
    endDateList[i]=y

In [ ]:
import requests
# Go through the helper lists and retrieve for every month the monthly global mean of 2m air temperature 
# and append a list of monthly global means.
monthlyAvgTS = []
for i,j in zip(startDateList,endDateList):
    url = url_fmt % ("temp2m", i.isoformat(),j.isoformat())
    r= requests.get(url,
                proxies={'http':None}
                )

    r.raise_for_status()
    k= [float(a) for a in r.text.split(",")]
    monthlyAvgTS.append(k[0])

In [ ]:
monthlyAvgTS

<hr>

#### EXTRA: Retrieve global monthly average 2m air temperature fields and store them into a multi-dimensional numpy array

WCS request of retrieving all global fields of one specific month and processing the monthly average of every grid-point on-the-fly. The query contains following processing steps: (1) Surface temperature values in Kelvin are converted to degree Celsius and (2) 6-hourly global surface temperature fields for one specific month are averaged.
* Access point: http://earthserver.ecmwf.int/rasdaman/ows?service=WCS&version=2.0.1
* Request type: &ProcessCoverages
* WCPS query:see below


In [ ]:
url_fmt_field = 'http://earthserver.ecmwf.int/rasdaman/ows?service=WCS&version=2.0.1' \
    '&request=ProcessCoverages&' \
    'query=for c in (%s) return encode((condense + over $x x(imageCrsDomain(c[Lat(%f:%f), Long(%f:%f),ansi("%s":"%s")], ansi))'\
    'using c[ansi($x)]/%d)-273.15, "csv")'

In [ ]:
# Create a raster stack of all monthly surface temperature averages
import numpy as np
monthlyStackGlobal = np.empty((361,720,12))
k = 0
for i,j in zip(startDateList,endDateList):
    slices = j.day*4
    url = url_fmt_field % ("temp2m", -90,90,-180,179.5, i.isoformat(),j.isoformat(), slices)
    r= requests.get(url,
                proxies={'http':None}
                )

    r.raise_for_status()
    x = np.array(eval(r.text.replace('{','[').replace('}',']')))
    monthlyStackGlobal[:,:,k] = x.transpose()
    k = k+1

In [ ]:
monthlyStackGlobal[:,:,8]

<hr>

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>